# Comenzando: Explorando los Fundamentos de NeMo

NeMo es una herramienta para crear aplicaciones de [IA Conversacional](https://developer.nvidia.com/conversational-ai#started).

El kit de herramientas NeMo permite a los investigadores componer fácilmente arquitecturas complejas de redes neuronales para IA conversacional utilizando componentes reutilizables, llamados Módulos Neuronales. Los Módulos Neuronales son bloques conceptuales de redes neuronales que toman entradas tipadas y producen salidas tipadas. Estos módulos generalmente representan capas de datos, codificadores, decodificadores, modelos de lenguaje, funciones de pérdida o métodos para combinar activaciones.

El kit incluye colecciones extensibles de módulos preconstruidos y modelos listos para usar en tareas como reconocimiento automático de voz (ASR), procesamiento de lenguaje natural (NLP) y síntesis de texto a voz (TTS). Diseñado para velocidad, NeMo puede aprovechar los Tensor Cores de NVIDIA y escalar el entrenamiento a múltiples GPUs y múltiples nodos.

Para más información, visita [la documentación de NeMo](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/#).

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install Cython packaging
!wget https://github.com/state-spaces/mamba/releases/download/v2.2.2/mamba_ssm-2.2.2+cu118torch2.1cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
!pip install mamba_ssm-2.2.2+cu118torch2.1cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
!pip install git+https://github.com/NVIDIA/NeMo-Run.git
!pip install git+https://github.com/NVIDIA/Megatron-LM#egg=megatron-core
!pip install nemo_toolkit['all']
## Install TorchAudio
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

## Grab the config we'll use in this example
!mkdir configs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
--2024-12-24 08:13:15--  https://github.com/state-spaces/mamba/releases/download/v2.2.2/mamba_ssm-2.2.2+cu118torch2.1cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/725839295/803cdf7f-7024-45d0-b544-9a0fde764302?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241224%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241224T081315Z&X-Amz-Expires=300&X-Am

mkdir: cannot create directory ‘configs’: File exists


## Fundamentos de NeMo
---------

Los modelos de NeMo se basan en el módulo [PyTorch Lightning](https://github.com/PyTorchLightning/pytorch-lightning) y son compatibles con todo el ecosistema de PyTorch. Esto significa que los usuarios tienen la flexibilidad completa de utilizar las APIs de alto nivel que proporciona PyTorch Lightning (a través de Trainer) o escribir sus propios bucles de entrenamiento y evaluación directamente en PyTorch (simplemente llamando al modelo y sus componentes individuales).

Para los desarrolladores de NeMo, un "Modelo" es la red neuronal (o redes neuronales) junto con toda la infraestructura que soporta dichas redes, empaquetada en una unidad singular y coherente. Por lo tanto, todos los modelos de NeMo están diseñados para incluir, como mínimo, los siguientes elementos listos para usar (algunos modelos también soportan funcionalidades adicionales):

 - **Arquitectura de red neuronal**: todos los módulos necesarios para el modelo.
 - **Conjuntos de datos y cargadores de datos**: todos los componentes que preparan los datos para su uso durante el entrenamiento o evaluación.
 - **Preprocesamiento y posprocesamiento**: todos los componentes que procesan los conjuntos de datos para que puedan ser consumidos fácilmente por los módulos.
 - **Optimizador y planificadores de tasa de aprendizaje**: configuraciones predeterminadas que funcionan de manera predeterminada y permiten experimentación adicional con facilidad.
 - **Cualquier otra infraestructura de soporte**: tokenizadores, configuración de modelos de lenguaje, aumento de datos, etc.

In [ ]:
import nemo
nemo.__version__

'2.0.0'

## Colecciones de NeMo

NeMo se divide en varias colecciones fundamentales según sus dominios: `asr`, `nlp`, `tts`. Cuando ejecutaste la instrucción `import nemo` mencionada anteriormente, ninguna de estas colecciones fue importada. Esto se debe a que es posible que no necesites todas las colecciones al mismo tiempo, por lo que NeMo permite importar parcialmente una o más colecciones según lo necesites.

-------
Importemos las tres colecciones mencionadas anteriormente:

In [ ]:
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
import nemo.collections.tts as nemo_tts

[NeMo W 2024-12-24 08:16:59 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/optimizer/__init__.py:17: UserWarning: Transformer Engine and Apex are not installed. Falling back to Torch optimizers.
      warnings.warn(
    
[NeMo W 2024-12-24 08:16:59 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/optimizer/optimizer.py:28: UserWarning: Transformer Engine and Apex are not installed. Falling back to local implementations of multi_tensor_applier and multi_tensor_scale
      warnings.warn(
    
[NeMo W 2024-12-24 08:16:59 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/optimizer/clip_grads.py:29: UserWarning: Transformer Engine and Apex are not installed. Falling back to local implementations of multi_tensor_applier, multi_tensor_l2norm, and multi_tensor_scale
      warnings.warn(
    
[NeMo W 2024-12-24 08:17:11 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/nemo/lightning/pytorch/strategies/utils.py:29: `t

## Modelos de NeMo en Colecciones

NeMo incluye varios modelos para cada una de sus colecciones, relacionados con tareas comunes en la IA conversacional. A continuación, echaremos un vistazo rápido a todos los modelos que NeMo ofrece para las 3 colecciones mencionadas anteriormente.

In [ ]:
asr_models = [model for model in dir(nemo_asr.models) if model.endswith("Model")]
asr_models

['ASRModel',
 'EncDecCTCModel',
 'EncDecClassificationModel',
 'EncDecDiarLabelModel',
 'EncDecFrameClassificationModel',
 'EncDecHybridRNNTCTCBPEModel',
 'EncDecHybridRNNTCTCModel',
 'EncDecK2RnntSeqModel',
 'EncDecK2SeqModel',
 'EncDecMultiTaskModel',
 'EncDecRNNTBPEModel',
 'EncDecRNNTModel',
 'EncDecSpeakerLabelModel',
 'SLUIntentSlotBPEModel',
 'SpeechEncDecSelfSupervisedModel']

In [ ]:
nlp_models = [model for model in dir(nemo_nlp.models) if model.endswith("Model")]
nlp_models

['BERTLMModel',
 'BertDPRModel',
 'BertJointIRModel',
 'DuplexDecoderModel',
 'DuplexTaggerModel',
 'DuplexTextNormalizationModel',
 'EntityLinkingModel',
 'GLUEModel',
 'IntentSlotClassificationModel',
 'MTEncDecModel',
 'MegatronGPTPromptLearningModel',
 'MultiLabelIntentSlotClassificationModel',
 'PunctuationCapitalizationLexicalAudioModel',
 'PunctuationCapitalizationModel',
 'QAModel',
 'SpellcheckingAsrCustomizationModel',
 'Text2SparqlModel',
 'TextClassificationModel',
 'ThutmoseTaggerModel',
 'TokenClassificationModel',
 'TransformerLMModel',
 'ZeroShotIntentModel']

In [ ]:
tts_models = [model for model in dir(nemo_tts.models) if model.endswith("Model")]
tts_models

['AlignerModel',
 'AudioCodecModel',
 'FastPitchModel',
 'GriffinLimModel',
 'HifiGanModel',
 'MelPsuedoInverseModel',
 'MixerTTSModel',
 'RadTTSModel',
 'SpectrogramEnhancerModel',
 'Tacotron2Model',
 'TwoStagesModel',
 'UnivNetModel',
 'VitsModel',
 'WaveGlowModel']

## El Modelo NeMo

Vamos a profundizar en lo que realmente es un modelo NeMo. Hay varias formas de crear estos modelos: podemos usar el constructor y pasar una configuración, instanciar el modelo desde un punto de control preentrenado, o simplemente proporcionar el nombre de un modelo preentrenado e instanciarlo directamente desde la nube.

---------
Por ahora, trabajemos con un modelo de ASR - [Citrinet](https://arxiv.org/abs/2104.01721).

In [ ]:
citrinet = nemo_asr.models.EncDecCTCModelBPE.from_pretrained('stt_en_citrinet_512')

[NeMo I 2024-12-24 08:17:38 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_512/versions/1.0.0rc1/files/stt_en_citrinet_512.nemo to /root/.cache/torch/NeMo/NeMo_2.0.0/stt_en_citrinet_512/3262321355385bb7cf5a583146117d77/stt_en_citrinet_512.nemo
[NeMo I 2024-12-24 08:17:40 common:826] Instantiating model from pre-trained checkpoint
[NeMo I 2024-12-24 08:17:45 mixins:173] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-12-24 08:17:45 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2024-12-24 08:17:45 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    
[NeMo W 2024-12-24 08:17:45 modelPT:189] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    T

[NeMo I 2024-12-24 08:17:45 features:305] PADDING: 16


[NeMo W 2024-12-24 08:17:47 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/nemo/core/connectors/save_restore_connector.py:682: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

[NeMo I 2024-12-24 08:17:47 save_restore_connector:275] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_2.0.0/stt_en_citrinet_512/3262321355385bb7cf5a583146117d77/stt_en_citrinet_512.nemo.


In [ ]:
citrinet.summarize()

  | Name              | Type                              | Params | Mode 
--------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | train
1 | encoder           | ConvASREncoder                    | 36.3 M | train
2 | decoder           | ConvASRDecoder                    | 657 K  | train
3 | loss              | CTCLoss                           | 0      | train
4 | spec_augmentation | SpectrogramAugmentation           | 0      | train
5 | wer               | WER                               | 0      | train
--------------------------------------------------------------------------------
37.0 M    Trainable params
0         Non-trainable params
37.0 M    Total params
147.977   Total estimated model params size (MB)
943       Modules in train mode
0         Modules in eval mode

## Configuración del Modelo usando OmegaConf
--------

¡Podemos descargar, instanciar y analizar la estructura de alto nivel del modelo `Citrinet` en unas pocas líneas! Ahora profundicemos en el archivo de configuración que hace que el modelo funcione.

Primero, importamos [OmegaConf](https://omegaconf.readthedocs.io/en/latest/). OmegaConf es una excelente biblioteca utilizada en todo NeMo que nos permite gestionar configuraciones en formato YAML de manera más sencilla. Además, funciona muy bien con otra biblioteca, [Hydra](https://hydra.cc/docs/intro/), que NeMo utiliza para realizar ediciones de configuración en tiempo real desde la línea de comandos, lo que mejora significativamente la facilidad de uso de nuestros archivos de configuración.

In [ ]:
from omegaconf import OmegaConf

Todos los modelos de NeMo vienen empaquetados con su configuración de modelo dentro del atributo `cfg`. Si bien técnicamente está diseñado para ser una declaración de configuración del modelo tal como se ha construido, `cfg` es una herramienta esencial para modificar el comportamiento del modelo después de su construcción. Esto facilita la realización de muchas tareas esenciales dentro de los modelos.

Para mayor seguridad, generalmente trabajamos con una copia de la configuración hasta que estamos listos para editarla dentro del modelo.

In [ ]:
import copy

In [ ]:
cfg = copy.deepcopy(citrinet.cfg)
print(OmegaConf.to_yaml(cfg))

sample_rate: 16000
train_ds:
  manifest_filepath: null
  sample_rate: 16000
  batch_size: 32
  trim_silence: true
  max_duration: 16.7
  shuffle: true
  is_tarred: false
  tarred_audio_filepaths: null
validation_ds:
  manifest_filepath: null
  sample_rate: 16000
  batch_size: 32
  shuffle: false
test_ds:
  manifest_filepath:
  - /home/smajumdar/PycharmProjects/nemo-eval/nemo_eval/librispeech/manifests/dev_other.json
  sample_rate: 16000
  batch_size: 32
  shuffle: false
  num_workers: 12
  pin_memory: true
model_defaults:
  repeat: 5
  dropout: 0.0
  separable: true
  se: true
  se_context_size: -1
tokenizer:
  dir: /home/smajumdar/PycharmProjects/nemo-eval/nemo_beta_eval/asrset/manifests/asrset_1.4/tokenizers/no_appen/tokenizer_spe_unigram_v1024/
  type: bpe
preprocessor:
  _target_: nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor
  sample_rate: 16000
  normalize: per_feature
  window_size: 0.025
  window_stride: 0.01
  window: hann
  features: 80
  n_fft: 512
  frame_s

## Analizando el contenido de la configuración del modelo
----------

Arriba vemos una configuración para el modelo Citrinet. Como se discutió al principio, los modelos de NeMo contienen la definición completa de la(s) red(es) neuronal(es), así como la mayor parte de la infraestructura circundante para soportar ese modelo dentro de sí mismos. Aquí vemos un ejemplo perfecto de este comportamiento.

La configuración de Citrinet incluye:

- **`preprocessor`**: Capa de preprocesamiento de MelSpectrogram.
- **`encoder`**: El modelo de codificador acústico.
- **`decoder`**: La capa de decodificador CTC.
- **`optim`** (y potencialmente **`sched`**): Configuración del optimizador. Opcionalmente, puede incluir información sobre el programador de tasas de aprendizaje.
- **`spec_augment`**: Soporte para la mejora de espectrogramas.
- **`train_ds`**, **`validation_ds`** y **`test_ds`**: Información para la construcción de conjuntos de datos y cargadores de datos.

## Modificando el contenido de la configuración del modelo
----------

Supongamos que queremos experimentar con un preprocesador diferente (queremos MelSpectrogram, pero con una configuración diferente a la proporcionada en la configuración original). O supongamos que queremos agregar un programador a este modelo durante el entrenamiento.

¡OmegaConf hace que esta tarea sea muy sencilla para nosotros!

In [ ]:
# OmegaConf won't allow you to add new config items, so we temporarily disable this safeguard.
OmegaConf.set_struct(cfg, False)

# Let's see the old optim config
print("Old Config: ")
print(OmegaConf.to_yaml(cfg.optim))

sched = {'name': 'CosineAnnealing', 'warmup_steps': 1000, 'min_lr': 1e-6}
sched = OmegaConf.create(sched)  # Convert it into a DictConfig

# Assign it to cfg.optim.sched namespace
cfg.optim.sched = sched

# Let's see the new optim config
print("New Config: ")
print(OmegaConf.to_yaml(cfg.optim))

# Here, we restore the safeguards so no more additions can be made to the config
OmegaConf.set_struct(cfg, True)

Old Config: 
name: novograd
lr: 0.05
betas:
- 0.8
- 0.25
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: 1000
  warmup_ratio: null
  min_lr: 1.0e-09
  last_epoch: -1

New Config: 
name: novograd
lr: 0.05
betas:
- 0.8
- 0.25
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: 1000
  min_lr: 1.0e-06



## Actualizando el modelo desde la configuración
----------

Los modelos de NeMo se pueden actualizar de varias maneras, pero seguimos patrones similares dentro de cada colección para mantener la coherencia.

Aquí mostraremos las dos formas más comunes de modificar los componentes principales del modelo: utilizando el método `from_config_dict` y actualizando algunas partes específicas del modelo.

Recuerda que todos los modelos de NeMo son módulos de PyTorch Lightning, los cuales a su vez son módulos de PyTorch, ¡así que tenemos mucha flexibilidad aquí!

### Actualizar el modelo utilizando `from_config_dict`

En ciertos archivos de configuración, notarás el siguiente patrón:

```yaml
preprocessor:
  _target_: nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor
  normalize: per_feature
  window_size: 0.02
  sample_rate: 16000
  window_stride: 0.01
  window: hann
  features: 64
  n_fft: 512
  frame_splicing: 1
  dither: 1.0e-05
  stft_conv: false
```

Podrías preguntarte, ¿por qué usamos `_target_`? Bueno, en general es poco común que el preprocesador, el codificador, el decodificador y quizás algunos otros detalles se cambien con frecuencia desde la línea de comandos al experimentar. Para estabilizar estas configuraciones, imponemos que nuestro preprocesador siempre sea del tipo `AudioToMelSpectrogramPreprocessor` para este modelo al establecer su atributo `_target_` en la configuración. Para proporcionar sus parámetros en el constructor de la clase, simplemente los agregamos después de `_target_`.

---------
Nota: aún podemos cambiar todos los parámetros de esta clase `AudioToMelSpectrogramPreprocessor` desde la línea de comandos utilizando Hydra, por lo que no perdemos flexibilidad una vez que decidimos qué tipo de clase de preprocesamiento queremos.

¡Esto también nos proporciona una forma flexible de instanciar partes del modelo solo a partir del objeto de configuración!

In [ ]:
new_preprocessor_config = copy.deepcopy(cfg.preprocessor)
new_preprocessor = citrinet.from_config_dict(new_preprocessor_config)
print(new_preprocessor)

NameError: name 'copy' is not defined

Entonces, ¿cómo actualizamos realmente el preprocesador interno de nuestro modelo con algo nuevo? Bueno, dado que los modelos de NeMo son simplemente módulos de PyTorch, ¡podemos reemplazar directamente su atributo!

In [ ]:
citrinet.preprocessor = new_preprocessor

In [ ]:
citrinet.summarize()

  | Name              | Type                              | Params | Mode 
--------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | train
1 | encoder           | ConvASREncoder                    | 36.3 M | train
2 | decoder           | ConvASRDecoder                    | 657 K  | train
3 | loss              | CTCLoss                           | 0      | train
4 | spec_augmentation | SpectrogramAugmentation           | 0      | train
5 | wer               | WER                               | 0      | train
--------------------------------------------------------------------------------
37.0 M    Trainable params
0         Non-trainable params
37.0 M    Total params
147.977   Total estimated model params size (MB)
943       Modules in train mode
0         Modules in eval mode

--------
Esto podría parecer que no cambió nada, ¡porque en realidad no modificamos la configuración del preprocesador en absoluto! Pero, como mostramos anteriormente, podemos modificar fácilmente la configuración del preprocesador, instanciarlo desde la configuración y luego simplemente asignarlo al modelo.

-------
**NOTA**: Los preprocesadores generalmente no tienen pesos, por lo que esto fue sencillo, pero, ¿qué pasa si queremos reemplazar una parte del modelo que realmente tiene parámetros entrenados?

Bueno, el enfoque anterior seguirá funcionando, solo recuerda que el nuevo módulo que insertes en `citrinet.encoder` o `citrinet.decoder` no tendrá pesos preentrenados. Sin embargo, puedes solucionar esto fácilmente cargando el estado del diccionario del módulo *antes* de asignarlo al modelo.

### Preservar la nueva configuración

Hemos actualizado el preprocesador del modelo. Sin embargo, también necesitamos realizar un paso crucial: **preservar la configuración actualizada**.

¿Por qué queremos hacer esto? NeMo tiene muchas formas de guardar y restaurar sus modelos, las cuales discutiremos más adelante. Todas ellas dependen de tener una configuración actualizada que defina el modelo en su totalidad, por lo que si modificamos algo, también debemos actualizar la parte correspondiente de la configuración para guardar y restaurar los modelos de manera segura.

In [ ]:
# Update the config copy
cfg.preprocessor = new_preprocessor_config
# Update the model config
citrinet.cfg = cfg

## Actualizar algunos componentes especiales del modelo
---------

Aunque el enfoque anterior es útil para la mayoría de los componentes principales del modelo, NeMo tiene utilidades especiales para algunos componentes específicos.

Estas utilidades son:

 - `setup_training_data`
 - `setup_validation_data` y `setup_multi_validation_data`
 - `setup_test_data` y `setup_multi_test_data`
 - `setup_optimization`

Estas utilidades especiales están diseñadas para ayudarte a configurar fácilmente el entrenamiento, la validación y las pruebas una vez que hayas restaurado un modelo desde un punto de control.

------
Una de las principales tareas de todos los modelos de IA conversacional es el ajuste fino en nuevos conjuntos de datos: nuevos idiomas, nuevos corpus de texto, nuevas voces, etc. A menudo no es suficiente contar solo con un modelo preentrenado. Por ello, se proporcionan estos métodos de configuración para permitir a los usuarios adaptar los modelos *después* de que ya han sido entrenados o entregados.




Es posible que recuerdes haber visto algunos mensajes de advertencia en el momento en que intentaste instanciar el modelo preentrenado. Esas advertencias, de hecho, son recordatorios para llamar a los métodos de configuración adecuados para la tarea que deseas realizar.

Esas advertencias simplemente muestran la configuración antigua que se utilizó para entrenar ese modelo y son una plantilla básica que puedes modificar fácilmente. Tienes la capacidad de modificar las subconfiguraciones `train_ds`, `validation_ds` y `test_ds` en su totalidad para evaluar, ajustar o entrenar el modelo desde cero, o para cualquier otro propósito que necesites.



Hablemos de cómo agregar un programador al modelo a continuación (que inicialmente solo tenía un optimizador en su configuración).

In [ ]:
# Let's print out the current optimizer
print(OmegaConf.to_yaml(citrinet.cfg.optim))

name: novograd
lr: 0.05
betas:
- 0.8
- 0.25
weight_decay: 0.001
sched:
  name: CosineAnnealing
  warmup_steps: 1000
  min_lr: 1.0e-06



In [ ]:
# Now let's update the config
citrinet.setup_optimization(cfg.optim)

[NeMo W 2024-12-24 08:19:21 modelPT:666] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2024-12-24 08:19:21 modelPT:787] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.25]
        eps: 1e-08
        grad_averaging: False
        lr: 0.05
        weight_decay: 0.001
    )


[NeMo W 2024-12-24 08:19:21 lr_scheduler:928] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.25]
     eps: 1e-08
     grad_averaging: False
     lr: 0.05
     weight_decay: 0.001
 ),
 None)

-------
Vemos una advertencia:

```
Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !
```

No tenemos un conjunto de datos de entrenamiento configurado, ni tampoco `max_steps` en la configuración. ¡La mayoría de los programadores de NeMo no pueden ser instanciados sin calcular cuántos pasos de entrenamiento existen realmente!

Aquí, podemos permitir temporalmente la construcción del programador pasando explícitamente un valor para `max_steps`, como 100.

In [ ]:
OmegaConf.set_struct(cfg.optim.sched, False)

cfg.optim.sched.max_steps = 100

OmegaConf.set_struct(cfg.optim.sched, True)

In [ ]:
# Now let's update the config and try again
citrinet.setup_optimization(cfg.optim)

[NeMo W 2024-12-24 08:19:28 modelPT:666] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2024-12-24 08:19:28 modelPT:787] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.25]
        eps: 1e-08
        grad_averaging: False
        lr: 0.05
        weight_decay: 0.001
    )
[NeMo I 2024-12-24 08:19:28 lr_scheduler:948] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7c332271ea40>" 
    will be used during training (effective maximum steps = 100) - 
    Parameters : 
    (warmup_steps: 1000
    min_lr: 1.0e-06
    max_steps: 100
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.25]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 0.05
     lr: 4.995004995004995e-05
     weight_decay: 0.001
 ),
 {'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7c332271ea40>,
  'interval': 'step',
  'frequency': 1,
  'monitor': 'loss',
  'reduce_on_plateau': False})

Podrías preguntarte por qué no configuramos explícitamente `citrinet.cfg.optim = cfg.optim`.

¡Esto se debe a que el método `setup_optimization()` lo hace por ti! Aún puedes actualizar la configuración manualmente.

### Configuración del Optimizador y el Planificador

Los optimizadores y los planificadores son componentes comunes de los modelos y son esenciales para entrenar el modelo desde cero.

Están agrupados bajo un espacio de nombres unificado `optim`, ya que los planificadores a menudo operan sobre un optimizador dado.


### Desglosemos la estructura de `optim`
```yaml
optim:
    name: novograd
    lr: 0.01

    # optimizer arguments
    betas: [0.8, 0.25]
    weight_decay: 0.001

    # scheduler setup
    sched:
      name: CosineAnnealing

      # Optional arguments
      max_steps: -1 # computed at runtime or explicitly set here
      monitor: val_loss
      reduce_on_plateau: false

      # scheduler config override
      warmup_steps: 1000
      warmup_ratio: null
      min_lr: 1e-9
```
Componentes esenciales del optimizador -

 - `name`: Nombre del optimizador como cadena de texto. Generalmente es el nombre de la clase en minúsculas.
 - `lr`: La tasa de aprendizaje es un argumento requerido para todos los optimizadores.

Componentes opcionales del optimizador - después de proporcionar los dos argumentos anteriores, cualquier argumento adicional agregado bajo `optim` se pasará al constructor de ese optimizador como argumentos de palabra clave.

 - `betas`: Lista de valores beta para pasar al optimizador.
 - `weight_decay`: Decaimiento de peso opcional pasado al optimizador.

Componentes opcionales del programador - `sched` es una configuración opcional del programador para el optimizador dado.

Si se proporciona `sched`, solo se necesita un argumento esencial:

 - `name`: El nombre del programador. Generalmente, es el nombre completo de la clase.

Componentes opcionales del programador -

 - `max_steps`: Máximo de pasos como una anulación del usuario. Si se proporciona `trainer.max_steps` dentro de la configuración del entrenador, se usa ese valor. Si no se establece ningún valor, el programador intentará calcular los `max_steps efectivos` utilizando el tamaño del cargador de datos de entrenamiento. Si eso también falla, entonces el programador no se creará en absoluto.

 - `monitor`: Se usa si estás utilizando un programador adaptativo como ReduceLROnPlateau. De lo contrario, se ignora. Por defecto es `loss`, indicando la pérdida de entrenamiento como monitor.

 - `reduce_on_plateau`: Necesario establecer en verdadero si se utiliza un programador adaptativo.

Cualquier argumento adicional bajo `sched` se suministrará como argumentos de palabra clave al constructor del programador.



## Diferencia entre los métodos de configuración del cargador de datos
----------

Podrás notar que tenemos múltiples métodos de configuración para los conjuntos de datos de validación y prueba. También notarás que no tenemos un equivalente `setup_multi_train_data`.

En general, los métodos `multi` se refieren a múltiples conjuntos de datos / cargadores de datos.*


### ¿Dónde está `setup_multi_train_data`?
Con lo anterior en mente, abordemos por qué no tenemos `setup_multi_train_data`.

NeMo se ocupa de múltiples dominios: `asr`, `nlp` y `tts`. La forma en que se configuran y utilizan los conjuntos de datos en estos dominios es dramáticamente diferente. A menudo no está claro qué significa tener múltiples conjuntos de datos de entrenamiento: ¿los concatenamos? ¿Los muestreamos aleatoriamente (con la misma o diferente probabilidad) de cada uno de ellos?

Por lo tanto, dejamos ese soporte para múltiples conjuntos de datos al propio modelo. Por ejemplo, en ASR, ¡puedes concatenar múltiples archivos de manifiesto de entrenamiento utilizando comas al proporcionar el valor de `manifest_filepath`!

### ¿Qué son los métodos multi?

En muchos casos, especialmente en ASR y NLP, podemos tener múltiples conjuntos de datos de validación y prueba. El ejemplo más común de esto en ASR es `Librispeech`, que tiene `dev_clean`, `dev_other`, `test_clean`, `test_other`.

NeMo estandariza cómo manejar múltiples cargadores de datos para validación y prueba, de modo que todas nuestras colecciones tengan una apariencia y sensación similar, así como facilitar el desarrollo de nuestros modelos. Durante la evaluación, estos conjuntos de datos se tratan de manera independiente y se les anteponen nombres resueltos para que los registros sean separados.

Por lo tanto, los métodos `multi` son generalizaciones de los métodos de configuración de datos de validación y prueba únicos, con alguna funcionalidad adicional. Si proporcionas múltiples conjuntos de datos, aún tienes que escribir código para solo un conjunto de datos y NeMo automáticamente adjuntará los nombres apropiados a tus registros para que puedas diferenciarlos.

Además, también preservan automáticamente la configuración que el usuario les pasa al actualizar los cargadores de datos de validación o prueba.

**En general, se prefiere llamar a los métodos `setup_multi_validation_data` y `setup_multi_test_data`, incluso si solo estás utilizando conjuntos de datos únicos, simplemente por la gestión automatizada que proporcionan.**

## Creación del Modelo desde el constructor vs restaurar un modelo
---------

Podrías notar que discutimos todos los métodos de configuración anteriores en el contexto del modelo después de que se ha restaurado. Sin embargo, los scripts de NeMo no los llaman dentro de ninguno de los scripts de entrenamiento de ejemplo.

Esto se debe a que estos métodos son llamados automáticamente por el constructor cuando el Modelo se crea por primera vez, pero estos métodos se omiten durante la restauración (ya sea desde un punto de control de PyTorch Lightning usando `load_from_checkpoint`, o mediante el método `restore_from` dentro de los Modelos de NeMo).

Esto se hace ya que la mayoría de los conjuntos de datos se almacenan en el directorio local de un usuario, y la ruta a estos conjuntos de datos se establece en la configuración (ya sea establecida por defecto, o establecida por sobrescrituras de Hydra). Por otro lado, los modelos están destinados a ser portátiles. En el sistema de otro usuario, los datos podrían no estar ubicados exactamente en el mismo lugar, o incluso en la misma unidad especificada en la configuración del modelo.

Por lo tanto, permitimos que el constructor tenga cierta flexibilidad y automatice dicha configuración de conjuntos de datos, mientras que la restauración advierte que los cargadores de datos no se configuraron y proporciona al usuario formas de configurar sus propios conjuntos de datos.

------

¿Por qué los optimizadores no se restauran automáticamente? Bueno, los optimizadores en sí mismos no enfrentan un problema, pero como vimos antes, los planificadores dependen del número de pasos de entrenamiento para calcular su programación.

Sin embargo, si no deseas modificar el optimizador y el planificador, y prefieres dejarlos en sus valores predeterminados, está perfectamente bien. ¡El método `setup_optimization()` es llamado automáticamente por PyTorch Lightning para ti cuando comienzas a entrenar tu modelo!

## Guardar y restaurar modelos
----------

NeMo proporciona varias formas de guardar y restaurar modelos. Si utilizas el Experiment Manager que forma parte de todos los scripts de entrenamiento de NeMo, PyTorch Lightning guardará automáticamente puntos de control en el directorio del experimento.

También podemos usar archivos empaquetados utilizando los métodos especializados `save_to` y `restore_from`.

### Guardar y Restaurar desde Checkpoints de PTL
----------

The PyTorch Lightning Trainer object will periodically save checkpoints when the experiment manager is being used during training.

PyTorch Lightning checkpoints can then be loaded and evaluated / fine-tuned just as always using the class method `load_from_checkpoint`.

For example, restore a Citrinet model from a checkpoint -

```python
citrinet = nemo_asr.models.EncDecCTCModelBPE.load_from_checkpoint(<path to checkpoint>)
```

### Guardar y Restaurar desde archivos .nemo
----------

Hay algunos modelos que pueden requerir dependencias externas para ser empaquetadas con ellos para restaurarlos correctamente.

Un ejemplo de esto es un modelo ASR con un tokenizador BPE externo. Es preferible que el modelo incluya todos los componentes necesarios para restaurarlo, pero un archivo binario para un tokenizador no puede ser serializado en un punto de control de PyTorch Lightning.

En tales casos, podemos usar el método `save_to` y `restore_from` para empaquetar todo el modelo y sus componentes (aquí, los archivos del tokenizador) en un archivo tar. Esto luego puede ser fácilmente importado por el usuario y utilizado para restaurar el modelo.

In [ ]:
# Save the model
citrinet.save_to('citrinet_512.nemo')

In [ ]:
!ls -d -- *.nemo

citrinet_512.nemo


In [ ]:
# Restore the model
temp_cn = nemo_asr.models.EncDecCTCModelBPE.restore_from('citrinet_512.nemo')

[NeMo I 2024-12-24 08:19:48 mixins:173] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-12-24 08:19:49 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2024-12-24 08:19:49 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    
[NeMo W 2024-12-24 08:19:49 modelPT:189] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    T

[NeMo I 2024-12-24 08:19:49 features:305] PADDING: 16
[NeMo I 2024-12-24 08:19:51 save_restore_connector:275] Model EncDecCTCModelBPE was successfully restored from /content/citrinet_512.nemo.


In [ ]:
temp_cn.summarize()

  | Name              | Type                              | Params | Mode 
--------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | train
1 | encoder           | ConvASREncoder                    | 36.3 M | train
2 | decoder           | ConvASRDecoder                    | 657 K  | train
3 | loss              | CTCLoss                           | 0      | train
4 | spec_augmentation | SpectrogramAugmentation           | 0      | train
5 | wer               | WER                               | 0      | train
--------------------------------------------------------------------------------
37.0 M    Trainable params
0         Non-trainable params
37.0 M    Total params
147.977   Total estimated model params size (MB)
943       Modules in train mode
0         Modules in eval mode

In [ ]:
# Note that the preprocessor + optimizer config have been preserved after the changes we made !
print(OmegaConf.to_yaml(temp_cn.cfg))

sample_rate: 16000
train_ds:
  manifest_filepath: null
  sample_rate: 16000
  batch_size: 32
  trim_silence: true
  max_duration: 16.7
  shuffle: true
  is_tarred: false
  tarred_audio_filepaths: null
validation_ds:
  manifest_filepath: null
  sample_rate: 16000
  batch_size: 32
  shuffle: false
test_ds:
  manifest_filepath:
  - /home/smajumdar/PycharmProjects/nemo-eval/nemo_eval/librispeech/manifests/dev_other.json
  sample_rate: 16000
  batch_size: 32
  shuffle: false
  num_workers: 12
  pin_memory: true
model_defaults:
  repeat: 5
  dropout: 0.0
  separable: true
  se: true
  se_context_size: -1
tokenizer:
  dir: /home/smajumdar/PycharmProjects/nemo-eval/nemo_beta_eval/asrset/manifests/asrset_1.4/tokenizers/no_appen/tokenizer_spe_unigram_v1024/
  type: bpe
  model_path: nemo:tokenizer.model
  vocab_path: nemo:vocab.txt
preprocessor:
  _target_: nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor
  sample_rate: 16000
  normalize: per_feature
  window_size: 0.025
  window_s

Nota: el archivo .nemo es un simple .tar.gz con el punto de control, la configuración y, potencialmente, otros artefactos como las configuraciones del tokenizador que está utilizando el modelo.

In [ ]:
!cp citrinet_512.nemo citrinet_512.tar.gz
!tar -xvf citrinet_512.tar.gz

./
./model_config.yaml
./model_weights.ckpt
./tokenizer.model
./vocab.txt


### Extrayendo puntos de control de PyTorch desde archivos tar de NeMo (Nivel de Modelo)
-----------

Si bien el archivo .nemo tar es una excelente manera de tener un modelo portátil, a veces es necesario que los investigadores tengan acceso al formato de guardado básico de PyTorch. NeMo tiene como objetivo ser completamente compatible con PyTorch y, por lo tanto, ofrece un método simple para extraer solo el punto de control de PyTorch desde el archivo .nemo tar.

In [ ]:
import torch

In [ ]:
state_dict = temp_cn.extract_state_dict_from('citrinet_512.nemo', save_dir='./pt_ckpt/')
!ls ./pt_ckpt/

[NeMo I 2024-12-24 08:20:10 save_restore_connector:346] Checkpoints from citrinet_512.nemo were successfully extracted into /content/pt_ckpt.
model_weights.ckpt


Como podemos ver a continuación, ahora hay un único punto de control básico de PyTorch disponible dentro del directorio `pt_ckpt`, que podemos usar para cargar los pesos de todo el modelo como se muestra a continuación.

In [ ]:
temp_cn.load_state_dict(torch.load('./pt_ckpt/model_weights.ckpt'))

[NeMo W 2024-12-24 08:20:15 nemo_logging:349] <ipython-input-28-85377080a5db>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
      temp_cn.load_state_dict(torch.load('./pt_c

<All keys matched successfully>

### Extrayendo puntos de control de PyTorch desde archivos tar de NeMo (Nivel de Módulo)
----------

Si bien el método anterior es excepcional para extraer el punto de control de todo el modelo, a veces puede ser necesario cargar y guardar los módulos individuales que componen el Modelo.

El mismo método de extracción ofrece una opción para extraer los puntos de control de nivel de modelo individual en sus archivos individuales, para que los usuarios tengan acceso a puntos de control por módulo.

In [ ]:
state_dict = temp_cn.extract_state_dict_from('citrinet_512.nemo', save_dir='./pt_module_ckpt/', split_by_module=True)
!ls ./pt_module_ckpt/

[NeMo I 2024-12-24 08:20:20 save_restore_connector:346] Checkpoints from citrinet_512.nemo were successfully extracted into /content/pt_module_ckpt.
decoder.ckpt  encoder.ckpt  preprocessor.ckpt


¡Ahora, podemos cargar y asignar los pesos de los módulos individuales del modelo Citrinet mencionado anteriormente!

In [ ]:
temp_cn.preprocessor.load_state_dict(torch.load('./pt_module_ckpt/preprocessor.ckpt'))
temp_cn.encoder.load_state_dict(torch.load('./pt_module_ckpt/encoder.ckpt'))
temp_cn.decoder.load_state_dict(torch.load('./pt_module_ckpt/decoder.ckpt'))

[NeMo W 2024-12-24 08:20:21 nemo_logging:349] <ipython-input-30-16927104504f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
      temp_cn.preprocessor.load_state_dict(torch

<All keys matched successfully>

# NeMo con Hydra

[Hydra](https://hydra.cc/docs/intro/) se utiliza en todo NeMo como una forma de habilitar la creación rápida de prototipos utilizando archivos de configuración predefinidos. Hydra y OmegaConf ofrecen una gran compatibilidad entre sí, y a continuación mostramos algunos consejos generales útiles para mejorar la productividad con Hydra al usar NeMo.

## Ayuda de Hydra
--------

Dado que nuestros scripts están escritos con Hydra en mente, es posible que notes que al usar `python <script.py> --help` te devuelve una configuración en lugar del formato de ayuda habitual de argparse.

Usando `--help` puedes ver la configuración predeterminada adjunta al script; cada script de NeMo tiene al menos un archivo de configuración predeterminado adjunto. Esto te da una guía sobre cómo puedes modificar valores para un experimento.

Hydra también tiene una bandera especial `--hydra-help`, que te ofrecerá más ayuda con respecto a Hydra en sí misma tal como está configurada en el script.


## Cambiando rutas y archivos de configuración
---------

Aunque todos los modelos de NeMo vienen con al menos 1 archivo de configuración predeterminado, uno podría querer cambiar las configuraciones sin cambiar el código. Esto se logra fácilmente con los siguientes comandos:

- `--config-path`: Ruta al directorio que contiene los archivos de configuración.
- `--config-name`: Nombre del archivo de configuración que deseamos cargar.

Tenga en cuenta que estos dos argumentos deben estar al principio de su declaración de ejecución, antes de proporcionar cualquier anulación de línea de comandos a su archivo de configuración.

## Sobrescribir la configuración desde la línea de comandos
----------

Hydra permite a los usuarios proporcionar sobrescrituras de línea de comandos a cualquier parte de la configuración. Hay tres casos a considerar:

 - Sobrescribir un valor existente en la configuración
 - Agregar un nuevo valor en la configuración
 - Eliminar un valor antiguo en la configuración

### Sobreescribiendo los valores existentes en config

Tomemos el caso en el que queremos cambiar el optimizador de `novograd` a `adam`. Cambiemos también los valores beta a los valores predeterminados de adam.

Las sobrescrituras de Hydra se basan en la sintaxis de `.` - cada `.` representa un nivel en la propia configuración.

```sh
$ python <script>.py \
  --config-path="dir to config" \
  --config-name="name of config" \
  model.optim.name="adam" \
  model.optim.betas=[0.9,0.999]
```

Cabe señalar que, si se pasan listas, no puede haber espacios entre los elementos.

------

También podemos admitir múltiples conjuntos de datos de validación con la sintaxis de lista anterior, pero depende del soporte a nivel de modelo.

Para la colección ASR, la siguiente sintaxis es ampliamente compatible en los modelos ASR, ASR-BPE y de clasificación. Tomemos un ejemplo de un modelo que se está entrenando en LibriSpeech -

```sh
$ python <script>.py \
  --config-path="dir to config" \
  --config-name="name of config" \
  model.validation_ds.manifest_filepath=["path to dev clean","path to dev other"] \
  model.test_ds.manifest_filepath=["path to test clean","path to test other"]
```

### Agregar nuevos valores en la configuración
----------

Hydra nos permite inyectar parámetros adicionales dentro de la configuración usando la sintaxis `+`.

Tomemos un ejemplo de agregar la corrección `amsgrad` para el optimizador `novograd` mencionado anteriormente.



### Eliminar valor antiguo en la configuración
---------

Hydra nos permite eliminar parámetros dentro de la configuración usando la sintaxis `~`.

Tomemos un ejemplo de eliminar `weight_decay` dentro del optimizador Novograd



## Fijando un valor a `None` desde la línea de comandos

Podemos optar por deshabilitar una característica estableciendo el valor en `None`.

Podemos lograr esto utilizando la palabra clave `null` en la línea de comandos.

Tomemos un ejemplo de deshabilitar el cargador de datos de validación dentro de la configuración de un modelo ASR -


```sh
$ python <script>.py \
  --config-path="dir to config" \
  --config-name="name of config" \
  model.test_ds.manifest_filepath=null
```

# Ejemplos de NeMo

NeMo admite varios modelos preconstruidos para tareas de ASR, NLP y TTS. Un ejemplo que vemos en este cuaderno es el modelo ASR para Conversión de Voz a Texto, utilizando el modelo Citrinet.

El repositorio de NeMo tiene un directorio dedicado `examples` con scripts para entrenar y evaluar modelos para varias tareas, que van desde ASR conversión de voz a texto, NLP respuesta a preguntas y TTS conversión de texto a voz utilizando modelos como `FastPitch` y `HiFiGAN`.

NeMo constantemente agrega nuevos modelos y nuevas tareas a estos ejemplos, de manera que estos ejemplos sirven como base para entrenar y evaluar modelos desde cero con los archivos de configuración proporcionados.

El directorio de Ejemplos de NeMo se puede encontrar aquí - https://github.com/NVIDIA/NeMo/tree/main/examples